In [2]:
root_dir = '/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/'
import sys
sys.path.append(root_dir)

import copy
import gc
import os
import argparse
import math
import time
import pickle
import re
import multiprocess as mp
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
sys.path.append("/share/tml_package")
from tml import utils
from scipy import io
from tqdm import tqdm
from collections import OrderedDict

import torch
from torch.nn import functional as F
from torch.nn import DataParallel as DP
from torch.utils.data import TensorDataset, DataLoader, Subset, Dataset
from accelerate import Accelerator
import pytorch_model_summary

import optuna
from optuna.samplers import RandomSampler, TPESampler
from optuna.integration import SkoptSampler

import numpy as np
import pandas as pd
import scipy.stats
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import matthews_corrcoef, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from data import TechDataset, CVSampler
from models import Transformer, Predictor
from train_utils import EarlyStopping, perf_eval, objective_cv, build_model, train_model, validate_model_mp
from utils import token2class, DotDict, to_device

from cleantext.sklearn import CleanTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# PART 1: Configuration

In [ ]:
mp.set_start_method("spawn")

In [3]:
args = argparse.Namespace(
    data_type="claim",
    data_file = "collection_[H01L,H10][2017].csv",
    target_ipc="H01L",
    pred_type="classification",
    n_TC = 5,
    use_pretrained_tokenizer=False,
    do_train=None,
    do_tune=None,
    n_folds=None,
    max_epochs=50,
    use_accelerator=None,
    do_save=False,
    n_gpus=3,
    light=True,
    config_file=os.path.join(root_dir, "configs", "USED_configs", "[CONFIGS]2023-04-11_00:39.json"),
    eval_train_set=False)

data_dir = os.path.join(root_dir, "data")
model_dir = os.path.join(root_dir, "models")
result_dir = os.path.join(root_dir, "results")
config_dir = os.path.join(root_dir, "configs")

if args.config_file is not None:
    config_file = args.config_file
else:
    config_file = os.path.join(config_dir, "configs_light.json") if args.light else os.path.join(config_dir, "configs.json")
configs = DotDict().load(config_file)
org_config_keys = {key: list(configs[key].keys()) for key in configs.keys()}

instant_configs = {key: value for (key, value) in vars(args).items() if value is not None} # if any argument passed when main.py executed
instant_configs_for_update = {configkey: {key: value for (key,value) in instant_configs.items() if key in org_config_keys[configkey]} for configkey in org_config_keys.keys()}
for key, value in configs.items():
    value.update(instant_configs_for_update[key])

regex_ipc = re.compile('[A-Z](?![\\D])')
if regex_ipc.match(configs.data.target_ipc) is None:
    configs.data.update({"target_ipc": "ALL"})
elif len(configs.data.target_ipc) > 5:
    configs.data.update({"target_ipc": configs.data.target_ipc[:4]})

if configs.model.model_type == "enc-pred-dec":
    configs.train.loss_weights["recon"] = configs.train.loss_weights["recon"] / sum(configs.train.loss_weights.values())
    configs.train.loss_weights["y"] = 1 - configs.train.loss_weights["recon"]
elif configs.model.model_type == "enc-pred":
    configs.train.loss_weights = {"recon": 0, "y": 1}
elif configs.model.model_type == "enc-dec":
    configs.train.loss_weights = {"recon": 1, "y": 0}

if configs.train.use_accelerator:
    accelerator = Accelerator()
    device_ids = list(range(torch.cuda.device_count()))
    device = accelerator.device

    configs.train.update({"accelerator": accelerator})
else:
    if torch.cuda.is_available():
        device_ids = list(range(torch.cuda.device_count()))
        gpu_usages = [np.sum([float(usage.split("uses")[-1].replace(" ","").replace("MB","")) for usage in torch.cuda.list_gpu_processes(id).split("GPU memory") if not usage=="" and "no processes are running" not in usage]) for id in device_ids]
        device_ids = np.argsort(gpu_usages)[:configs.train.n_gpus]
        device_ids = list(map(lambda x: torch.device('cuda', x),list(device_ids)))
        device = device_ids[0] # main device
        torch.cuda.set_device(device)
    else:
        device = torch.device('cpu')
        device_ids = []

configs.data.update({"root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "result_dir": result_dir})
configs.train.update({"device": device,
                        "device_ids": device_ids,
                        "root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "use_keywords": configs.data.use_keywords,
                        "early_stop_patience": int(0.3*configs.train.max_epochs)})
configs.model.update({"device": device,
                        "device_ids": device_ids,
                        "n_directions": 2 if configs.model.bidirec else 1,
                        "use_accelerator": configs.train.use_accelerator})

## Set hyperparameters for model training (To be TUNED)
if configs.train.do_train and configs.train.do_tune:
    n_layers = configs.model.n_layers = None
    d_embedding = configs.model.d_embedding = None
    d_hidden = configs.model.d_hidden = None
    d_latent = None
    learning_rate = configs.train.learning_rate = None
    batch_size = configs.train.batch_size = None
    config_name = "HPARAM_TUNING"
    final_model_path = None
else:
    n_layers = configs.model.n_layers
    d_embedding = configs.model.d_embedding
    d_hidden = configs.model.d_hidden
    d_latent = configs.model.n_enc_seq * configs.model.d_hidden

    key_components = {"data": ["target_ipc", "pred_type", "max_seq_len", "vocab_size"], "model": ["n_layers", "d_hidden", "d_embedding", "d_ff", "n_head", "d_head"], "train": ["learning_rate", "batch_size", "max_epochs"]}
    config_name = ""
    for key in key_components.keys():
        for component in key_components[key]:
            config_name += "["+str(configs[key][component])+component+"]"
    final_model_path = os.path.join(model_dir, f"[Final_model]{config_name}.ckpt")

configs.model.update({"d_latent": d_latent})
configs.train.update({"config_name": config_name,
                        "final_model_path": final_model_path})


# PART 2: Dataset setting

In [4]:
tstart = time.time()
org_config_keys_temp = copy.copy(org_config_keys["data"])
org_config_keys_temp.pop(org_config_keys_temp.index("use_pretrained_tokenizer"))
org_config_keys_temp.pop(org_config_keys_temp.index("data_file"))
dataset_config_name = "-".join([str(key)+"="+str(value) for (key,value) in configs.data.items() if key in org_config_keys_temp])
dataset_path = os.path.join(data_dir, "pickled_dataset", "[tech_dataset]"+dataset_config_name+".pickle")
if os.path.exists(dataset_path) and args.do_save is False:
    print("Load pickled dataset...")
    with open(dataset_path, "rb") as f:
        tech_dataset = pickle.load(f)   # Load pickled dataset if dataset with same configuration already saved
    print("Pickled dataset loaded")
else:
    print("Make dataset...")
    tech_dataset = TechDataset(configs.data)
    with open(dataset_path, "wb") as f:
        tech_dataset.rawdata = None
        pickle.dump(tech_dataset, f)
tend = time.time()
print(f"{np.round(tend-tstart,4)} sec elapsed for loading patents for class [{configs.data.target_ipc}]")

configs.model.update({"tokenizer": tech_dataset.tokenizer,
                        "n_enc_vocab": tech_dataset.tokenizer.get_vocab_size(),
                        "n_dec_vocab": tech_dataset.tokenizer.get_vocab_size(),
                        "n_enc_seq": tech_dataset.max_seq_len,
                        "n_dec_seq": tech_dataset.max_seq_len,
                        "n_outputs": 1 if configs.data.pred_type=="regression" else tech_dataset.n_outputs,
                        "i_padding": tech_dataset.tokenizer.token_to_id("<PAD>")})
configs.model.update({"d_latent": configs.model.n_enc_seq * configs.model.d_hidden})

Load pickled dataset...
Pickled dataset loaded
0.2392 sec elapsed for loading patents for class [H01L]


In [7]:
result_path = os.path.join(root_dir, "results")

In [11]:
train_data = pd.read_excel(os.path.join(result_path, "[DATASET]2023-04-11_00:39.xlsx"), sheet_name="TRAIN_dataset")
test_data = pd.read_excel(os.path.join(result_path, "[DATASET]2023-04-11_00:39.xlsx"), sheet_name="TEST_dataset")

In [15]:
tech_dataset.data.loc[test_data["number"]]

,main ipc,sub ipc,claims,TC5,TC5_digitized,class
number,,,,,,
9673291,H01L29/45,H01L27/1157;H01L27/11582,"1. A semiconductor memory device, comprising: ...",0,0,106
9676778,A61K31/4985,C07D487/04;C07D471/04;A61K45/06,1. A compound of Formula (I): ##STR00016## o...,0,0,11
9759818,G01T1/20,G01T1/24;H01L27/146,"1. A flat panel X-ray detector, comprising: a ...",0,0,94
9786666,H01L27/092,H01L21/8238;H01L29/66;H01L29/78;H01L29/161;H01...,1. A semiconductor structure comprising: a sil...,1,0,106
9673259,H01L27/30,H01L51/00;H01L51/42,"1. An organic photoelectronic device, comprisi...",2,0,106
...,...,...,...,...,...,...
9589954,H01L27/088,H01L29/78;H01L21/762;H01L21/32;H01L21/311;H01L...,"1. A semiconductor device, comprising: a subst...",2,0,106
9605099,C08F210/16,C08L23/00;C08F10/00;C08J5/18;H01L31/0203,1. A polyolefin resin for an encapsulant for a...,0,0,53
9768117,H01L23/528,H01L27/1157;H01L27/11582;H01L27/11524;H01L27/1...,"1. A semiconductor device, comprising: a subst...",2,0,106


# PART 3: Training

In [ ]:
sampler = CVSampler(tech_dataset, n_folds=configs.train.n_folds, test_ratio=0.1, stratify=True)
cv_idx = sampler.get_idx_dict()
print(f"#Samples\nTrain: {len(cv_idx[0]['train'])}, Validation: {len(cv_idx[0]['val'])}, Test: {len(cv_idx[0]['test'])}")

## PART 3-1: Hyperparameter tuning

In [ ]:
if configs.train.do_tune:
    configs.train.update({"tuned_model_path": os.path.join(model_dir,"hparam_tuning")})
    optuna_obj = lambda trial: objective_cv(trial, dataset=tech_dataset, cv_idx=cv_idx, model_params=configs.model, train_params=configs.train)

    opt_sampler = TPESampler()
    study = optuna.create_study(direction='minimize')
    study.optimize(optuna_obj, n_trials=configs.train.n_trials, gc_after_trial=True)
    best_params = study.best_trial.params

    print(f"Best trial:\n  CrossEntropyLoss: {study.best_trial.value}\n  Params:")
    for k, v in best_params.items():
        print(f"    {k}: {v}")

    configs.train.update({k: v for k,v in best_params.items() if k in configs.train.keys()})
    configs.model.update({k: v for k,v in best_params.items() if k in configs.model.keys()})
    config_name = f"{configs.model.n_layers}layers_{configs.model.d_embedding}emb_{configs.model.d_hidden}hid_{configs.model.n_directions}direc_{np.round(configs.train.learning_rate, 4)}lr_{configs.train.batch_size}batch_{configs.train.max_epochs}ep"
    final_model_path = os.path.join(model_dir, f"[Final_model][{configs.data.target_ipc}]{config_name}.ckpt")

## PART 3-2: Dataset construction and model training

In [ ]:
## Construct datasets
train_idx = cv_idx[0]['train']
val_idx = cv_idx[0]['val']
test_idx = cv_idx[0]['test']
whole_idx = np.concatenate([train_idx, val_idx])

train_dataset = Subset(tech_dataset, train_idx)
val_dataset = Subset(tech_dataset, val_idx)
test_dataset = Subset(tech_dataset, test_idx)
whole_dataset = Subset(tech_dataset, whole_idx)

train_loader = DataLoader(train_dataset, batch_size=configs.train.batch_size, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=configs.train.batch_size if len(val_idx)>configs.train.batch_size else len(val_idx), shuffle=True, num_workers=4, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=configs.train.batch_size if len(test_idx)>configs.train.batch_size else len(test_idx), shuffle=False, num_workers=4)
whole_loader = DataLoader(whole_dataset, batch_size=configs.train.batch_size, shuffle=False, num_workers=4)

## Load best model or build model
final_model = build_model(configs.model, tokenizer=tech_dataset.tokenizer)

In [ ]:
with torch.no_grad():
    x_input, *_ = tech_dataset.__getitem__(0)
    x_input = torch.tensor(x_input, device=device).unsqueeze(0)
    print(pytorch_model_summary.summary(final_model.module, torch.zeros(x_input.shape, device=device, dtype=torch.long), torch.zeros(x_input.shape, device=device, dtype=torch.long), show_input=True, max_depth=None, show_parent_layers=True))

In [ ]:
## Train model
if configs.train.do_tune:
    best_states = torch.load(os.path.join(configs.train.tuned_model_path,f"[HPARAM_TUNING]{study.best_trial.number}trial.ckpt"))
    converted_states = OrderedDict()
    for k, v in best_states.items():
        if 'module' not in k:
            k = 'module.'+k
        else:
            k = k.replace('features.module.', 'module.features.')
        converted_states[k] = v
    final_model.load_state_dict(converted_states)
else:
    final_model = train_model(final_model, train_loader, val_loader, configs.model, configs.train)
torch.save(final_model.state_dict(), final_model_path) # Finalize
torch.cuda.empty_cache()

In [ ]:
X, X_keywords, Y = next(iter(train_loader))
src, trg, y = X_keywords.to(device), X.to(device), Y.to(device)

In [ ]:
trg

In [ ]:
y

In [ ]:
tech_dataset.n_outputs

In [ ]:
outputs = final_model(src, trg[:,:-1])

In [ ]:
outputs_recon = [output[0] for output in outputs] # outputs_recon: n_gpus *

## PART 3-3: Training evaluation

In [ ]:
if args.eval_train_set:
    ## Evaluation on train dataset
    print("Validate model on train dataset")
    # trues_recon_train, preds_recon_train, trues_y_train, preds_y_train = validate_model(final_model, whole_loader, configs.model, configs.train)
    val_res_train = validate_model_mp(final_model, whole_dataset, mp=mp, model_params=configs.model, train_params=configs.train)
    trues_recon_train = np.concatenate([res["recon"]["true"] for res in val_res_train.values()])
    preds_recon_train = np.concatenate([res["recon"]["pred"] for res in val_res_train.values()])
    trues_y_train = np.concatenate([res["y"]["true"] for res in val_res_train.values()])
    preds_y_train = np.concatenate([res["y"]["pred"] for res in val_res_train.values()])

    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_y_train = perf_eval("TRAIN_SET", trues_y_train, preds_y_train, configs=configs, pred_type=configs.data.pred_type)
    if configs.data.pred_type == "classification":
        eval_y_train, confmat_y_train = eval_y_train
else:
    eval_recon_train = eval_y_train = None

## Evaluation on test dataset
print("Validate model on test dataset")
# trues_recon_test, preds_recon_test, trues_y_test, preds_y_test = validate_model(final_model, test_loader, configs.model, configs.train)
val_res_test = validate_model_mp(final_model, test_dataset, mp=mp, batch_size=64, model_params=configs.model, train_params=configs.train)
trues_recon_test = np.concatenate([res["recon"]["true"] for res in val_res_test.values()])
preds_recon_test = np.concatenate([res["recon"]["pred"] for res in val_res_test.values()])
trues_y_test = np.concatenate([res["y"]["true"] for res in val_res_test.values()])
preds_y_test = np.concatenate([res["y"]["pred"] for res in val_res_test.values()])

eval_recon_test = perf_eval("TEST_SET", trues_recon_test, preds_recon_test, configs=configs,  pred_type='generative', tokenizer=final_model.module.tokenizer)
eval_y_test = perf_eval("TEST_SET", trues_y_test, preds_y_test, configs=configs, pred_type=configs.data.pred_type)
if configs.data.pred_type == "classification":
    eval_y_test, confmat_y_test = eval_y_test

eval_recon_res = pd.concat([eval_recon_train, eval_recon_test], axis=0)
eval_y_res = pd.concat([eval_y_train, eval_y_test], axis=0)
if configs.data.pred_type == "classification":
    confmat_y_res = pd.concat([confmat_y_train, confmat_y_test], axis=0)


In [ ]:
raise

# (temp) Pre-trained model experiment

In [ ]:
from transformers import BartTokenizer, BartModel, BartForConditionalGeneration

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)
    
    def forward(self, input_ids, mask):
        x = self.l1(input_ids=input_ids, attention_mask=mask)
        hidden_state = x[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        
        return output, hidden_state

In [ ]:
m = BERTClass().to(device)

In [ ]:
out = tokenizer(tech_dataset.data['claims'][:100].tolist(), add_special_tokens=True, max_length=256, padding="max_length", truncation=True)

In [ ]:
inputs = {"input_ids": torch.tensor(out.input_ids, dtype=torch.long, device=device),
         "mask": torch.tensor(out.attention_mask, dtype=torch.long, device=device)}

In [ ]:
temp = m(inputs["input_ids"], inputs["mask"])

In [ ]:
temp[1].shape

In [ ]:
device = torch.device("cuda", 0)

In [ ]:
print(pytorch_model_summary.summary(m.to(device), torch.zeros(inputs["input_ids"].shape, device=device, dtype=torch.long), torch.zeros(inputs["input_ids"].shape, device=device, dtype=torch.long), show_input=True, max_depth=None, show_parent_layers=True))

## Inference

In [ ]:
final_model_path

In [ ]:
final_model = build_model(configs.model)
if os.path.exists(final_model_path):
    best_states = torch.load(final_model_path)
else:
    raise Exception("Model need to be trained first")
converted_states = OrderedDict()
for k, v in best_states.items():
    if 'module' not in k:
        k = 'module.'+k
    else:
        k = k.replace('features.module.', 'module.features.')
    converted_states[k] = v
final_model.load_state_dict(converted_states)

In [ ]:
# Copy predictor
temp_path = os.path.join(model_dir, "temp", "temp.ckpt")
predictor = Predictor(final_model.module.config).to(final_model.module.device)
torch.save(final_model.module.predictor.state_dict(), temp_path)
predictor.load_state_dict(torch.load(temp_path, map_location=final_model.module.device))

In [ ]:
list(predictor.parameters())[5].shape

In [ ]:
inputs = enc_outputs[:2]

In [ ]:
def get_tensor_info(tensor):
  info = []
  for name in ['requires_grad', 'is_leaf', 'retains_grad', 'grad_fn', 'grad']:
    info.append(f'{name}({getattr(tensor, name, None)})')
  info.append(f'tensor({str(tensor)})')
  return ' '.join(info)

In [ ]:
inputs.retain_grad()

In [ ]:
print(get_tensor_info(inputs))

In [ ]:
predictor(inputs)

In [ ]:
predictor(inputs)[0,-1,-1].backward()

In [ ]:
print(get_tensor_info(inputs))

In [ ]:
instant_dataset = Subset(tech_dataset, np.random.choice(np.arange(len(tech_dataset)), 1000))
data_loader = DataLoader(instant_dataset, batch_size=16)

In [ ]:
class_weights = np.unique(tech_dataset.Y[whole_idx], return_counts=True)[1]

In [ ]:
batch_data = next(iter(data_loader))

In [ ]:
text_inputs = to_device(batch_data["text_inputs"], device)

In [ ]:
enc_outputs, *_ = final_model.module.encoder(**text_inputs)

In [ ]:
enc_outputs.shape

In [ ]:
z = enc_outputs[:,-1,:]

In [ ]:
preds = final_model.module.predictor(z)

In [ ]:
preds.shape

In [ ]:
torch.tensor(1).item()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
preds_recon = []

if str(configs.model["tokenizer"].__class__).split("\'")[1].split(".")[0] == "transformers":
    preds_recon_batch = torch.tile(torch.tensor(configs.model['tokenizer'].convert_tokens_to_ids("<SOS>"), device=device), dims=(text_inputs["input_ids"].shape[0],1)).to(device=device)
elif str(configs.model["tokenizer"].__class__).split("\'")[1].split(".")[0] == "tokenizers":
    preds_recon_batch = torch.tile(torch.tensor(configs.model['tokenizer'].token_to_id("<SOS>"), device=device), dims=(text_inputs["input_ids"].shape[0],1)).to(device=device)
    
for i in range(configs.model['n_dec_seq']-1):
    dec_outputs, *_ = final_model.module.decoder(preds_recon_batch, text_inputs["input_ids"], enc_outputs)
    pred_tokens = dec_outputs.argmax(2)[:,-1].unsqueeze(1)
    preds_recon_batch = torch.cat([preds_recon_batch, pred_tokens], axis=1)
    torch.cuda.empty_cache()
preds_recon.append(preds_recon_batch[:,1:].cpu().detach().numpy())
preds_recon = np.concatenate(preds_recon)

In [ ]:
display("TRUE(keywords): "+pd.Series(configs.model.tokenizer.decode_batch(batch_data["text_inputs"]["input_ids"].cpu().detach().numpy()))[0])

display("TRUE(org): "+pd.Series(configs.model.tokenizer.decode_batch(batch_data["text_outputs"]["input_ids"].cpu().detach().numpy()))[0])

display("PRED: "+pd.Series(configs.model.tokenizer.decode_batch(preds_recon, skip_special_tokens=False)).apply(lambda x: x.split("<EOS>")[0])[0])

In [ ]:
enc_outputs_ = torch.normal(enc_outputs, torch.tile(torch.tensor(2), dims=enc_outputs.size()).to(device))

In [ ]:
preds_recon_ = []

if str(configs.model["tokenizer"].__class__).split("\'")[1].split(".")[0] == "transformers":
    preds_recon_batch = torch.tile(torch.tensor(configs.model['tokenizer'].convert_tokens_to_ids("<SOS>"), device=device), dims=(text_inputs["input_ids"].shape[0],1)).to(device=device)
elif str(configs.model["tokenizer"].__class__).split("\'")[1].split(".")[0] == "tokenizers":
    preds_recon_batch = torch.tile(torch.tensor(configs.model['tokenizer'].token_to_id("<SOS>"), device=device), dims=(text_inputs["input_ids"].shape[0],1)).to(device=device)
    
for i in range(configs.model['n_dec_seq']-1):
    dec_outputs, *_ = final_model.module.decoder(preds_recon_batch, text_inputs["input_ids"], enc_outputs_)
    pred_tokens = dec_outputs.argmax(2)[:,-1].unsqueeze(1)
    preds_recon_batch = torch.cat([preds_recon_batch, pred_tokens], axis=1)
    torch.cuda.empty_cache()
preds_recon_.append(preds_recon_batch[:,1:].cpu().detach().numpy())
preds_recon_ = np.concatenate(preds_recon_)

In [ ]:
i = 10

print("TRUE(keywords): "+pd.Series(configs.model.tokenizer.decode_batch(batch_data["text_inputs"]["input_ids"].cpu().detach().numpy()))[i])

print("TRUE(org): "+pd.Series(configs.model.tokenizer.decode_batch(batch_data["text_outputs"]["input_ids"].cpu().detach().numpy()))[i])

print("PRED: "+pd.Series(configs.model.tokenizer.decode_batch(preds_recon, skip_special_tokens=False)).apply(lambda x: x.split("<EOS>")[0])[i])

print("PRED(modified): "+pd.Series(configs.model.tokenizer.decode_batch(preds_recon_, skip_special_tokens=False)).apply(lambda x: x.split("<EOS>")[0])[i])